<a href="https://colab.research.google.com/github/hiranlowe/FitionModel/blob/main/Fition_Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries




In [ ]:

import os
from google.colab import drive

import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv

from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation, Bidirectional, Conv1D, MaxPool1D, BatchNormalization, Dense, LSTM

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

Initialize TPU

In [ ]:
# Function to get hardware strategy
def get_hardware_strategy():
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        tf.config.optimizer.set_jit(True)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    return tpu, strategy

tpu, strategy = get_hardware_strategy()


Running on TPU  grpc://10.69.168.146:8470
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.69.168.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.69.168.146:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


CSI packet processing function

In [ ]:
def _read_csi_next(pcapfile, csi_size):
    """
    Note: Designed for internal use only.
    
    Parameters
    ----------
        pcapfile : File Object
        csi_size : Expected length of CSI in bytes. NFFT * 4
    """

    # Read Frame Size
    pcapfile.seek(8, os.SEEK_CUR)
    frame_size = int.from_bytes(
        pcapfile.read(4),
        byteorder = 'little',
        signed = False
    )

    # Skip some stuff
    pcapfile.seek(56, os.SEEK_CUR)

    # Read CSI data
    pcapfile.seek(8, os.SEEK_CUR)
    csi = np.frombuffer(
        pcapfile.read(csi_size), 
        dtype = np.int16,
        count = int(csi_size / 2)
    )

    # Skip any zero-padding
    pcapfile.seek((frame_size - csi_size - 60), os.SEEK_CUR)

    return csi

def read_csi(pcap_file_path):
    """
    You have to remove 
    null and pilot subcarriers
    yourself.

    Parameters
    ----------
        pcap_file_path : str
    """

    bandwidth = 80

    NFFT = int(bandwidth * 3.2) # Number of channels in FFT
    chunksize = 1024

    csi  = np.zeros((chunksize, NFFT * 2), dtype = 'int16')

    with open(pcap_file_path, 'rb') as pcapfile:
        filesize = os.stat(pcap_file_path).st_size
        pcapfile.seek(24, os.SEEK_SET)

        npackets = 0
        while pcapfile.tell() < filesize:
            if not (npackets % chunksize):
                csi = np.vstack((csi, np.zeros((chunksize, NFFT * 2), dtype = 'int16')))

            csi[npackets] = _read_csi_next(pcapfile, NFFT * 4)
            
            npackets += 1

    # Convert CSI complex numbers to Magnitude.
    csi_converted = np.abs(
        np.fft.fftshift(csi[:npackets, ::2] + 1.j * csi[:npackets, 1::2], axes=(1,))
    )

    return csi_converted

Processing data set files


> Import files -> call the above function -> remove unrelevant subcarriers 



In [ ]:
drive.mount("/content/gdrive")

movement_data=[]
values={0:"nm",1:"sitdown", 2:"standup",3:"walking",4:"falling",5:"getintobed"}
for key, value in values.items():
    for i in range(1,31):
        for j in range(1,5):
            file_path = "/content/gdrive/My Drive/Fition Dataset/"+value+"_user"+str(j)+"_rm"+str(j)+"/"+value+"_user"+str(j)+"_rm"+str(j)+"_"+str(i)+".pcap"
            # print(file_path)
            csi = read_csi(file_path)
        # 80 MHz
            nullsubcarriers  = np.array([x+128 for x in [-128, -127, -126, -125, -124, -123, -1, 0, 1, 123, 124, 125, 126, 127]])
            pilotsubcarriers = np.array([x+128 for x in [-103, -75, -39, -11, 11, 39, 75, 103]])

        # 40 MHz
        #  nullsubcarriers = [x+64 for x in [-64, -63, -62, -61, -60, -59, -1, 0, 1, 59, 60, 61, 62, 63]]
        #  pilotsubcarriers = [x+64 for x in [11, 25, 53, -11, -25, -53]]

            csi=np.delete(csi,np.s_[nullsubcarriers],axis=1)
            csi=np.delete(csi,np.s_[pilotsubcarriers],axis=1)
            movement_data.append(csi)
        
   

Mounted at /content/gdrive


Shape of the input

In [ ]:
x=np.array(movement_data)
x.shape

(720, 500, 234)

Creating train, validation and test inputs

In [ ]:
x_train = []
x_valid = []
x_test = []
ind_list = []
for i in range(0, 720, 6):
    ind_list.append(i)
    # print(len(ind_list))
for i in range(0, 720):
    if(i in ind_list):
        x_test.append(x[i])
    # elif(i-1 in ind_list):
    #     x_valid.append(x[i])
        # print(i)
    else:
        x_train.append(x[i])



x_train = np.array(x_train)
x_valid = np.array(x_valid)
x_test = np.array(x_test)
print(len(x_train))
print(len(x_valid))
print(len(x_test))

600
0
120


Creating train, validation and test labels

In [ ]:
list1=[0 for _ in range(100)]
list2=[1 for _ in range(100)]
list3=[2 for _ in range(100)]
list4=[3 for _ in range(100)]
list5=[4 for _ in range(100)]
list6=[5 for _ in range(100)]
y_train=np.array(list1+list2+list3+list4+list5+list6)
#y_train=y_train.reshape(20)
y_train.shape


(600,)

In [ ]:
list1=[0 for _ in range(20)]
list2=[1 for _ in range(20)]
list3=[2 for _ in range(20)]
list4=[3 for _ in range(20)]
list5=[4 for _ in range(20)]
list6=[5 for _ in range(20)]
y_valid=np.array(list1+list2+list3+list4+list5+list6)
#y_test=y_test.reshape(10,1)
y_valid.shape

(120,)

In [ ]:
list1=[0 for _ in range(20)]
list2=[1 for _ in range(20)]
list3=[2 for _ in range(20)]
list4=[3 for _ in range(20)]
list5=[4 for _ in range(20)]
list6=[5 for _ in range(20)]
y_test=np.array(list1+list2+list3+list4+list5+list6)
#y_test=y_test.reshape(10,1)
y_test.shape


(120,)

Encoding Labels

In [ ]:
y_train=to_categorical(y_train)
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [ ]:
y_train.shape

(600, 6)

In [ ]:
y_valid.shape


(120, 6)

In [ ]:
y_test.shape


(120, 6)

Input shape variables

In [ ]:
input_length, input_features = 500, 234

Optuna tuning

In [ ]:
def create_model(trial):

    x0 = tf.keras.layers.Input(shape=(n_length, n_features))  

    #conv layers
    conv_layers = 2
    conv_filters = trial.suggest_int("conv_filters_L1", 100, 150)
    conv_kernels = trial.suggest_int("conv_kernel_L1", 2, 6)
    activation = trial.suggest_categorical("activation_L1", ["relu", "selu", "elu", "swish", "tanh"])
    conv = Conv1D(filters = conv_filters, kernel_size = conv_kernels)(x0)
    conv = BatchNormalization()(conv)
    conv = Activation(activation)(conv)
    conv = BatchNormalization()(conv)
    for i in range(conv_layers-1):
        conv_filters = trial.suggest_int("conv_filters_L{}".format(i+2), 100, 150)
        conv_kernels = trial.suggest_int("conv_kernel_L{}".format(i+2), 2, 6)
        activation = trial.suggest_categorical("activation_L{}".format(i+2), ["relu", "selu", "elu", "swish", "tanh"])
        conv = Conv1D(filters = conv_filters, kernel_size = conv_kernels)(conv)
        conv = BatchNormalization()(conv)
        conv = Activation(activation)(conv)
        conv = BatchNormalization()(conv)

    #maxpool
    maxp = MaxPool1D()(conv)

    #lstm layers
    lstm_layers = 4
    lstm_units = trial.suggest_int("lstm_units_L1", 100, 500)
    lstm = Bidirectional(keras.layers.LSTM(lstm_units, return_sequences=True))(maxp)
    for i in range(lstm_layers-2):
        lstm_units = trial.suggest_int("lstm_units_L{}".format(i+2), 100, 500)
        lstm = Bidirectional(keras.layers.LSTM(lstm_units, return_sequences=True))(lstm)    
    lstm_units = trial.suggest_int("lstm_units_L4", 100, 500)
    lstm = Bidirectional(keras.layers.LSTM(lstm_units))(lstm)
    # dropout_rate = trial.suggest_float("lstm_dropout", 0.0, 0.3)
    # lstm = Dropout(dropout_rate)(lstm)
    # dense_units = lstm_units[-1]
    
    
    # lstm = Dense(dense_units, activation=activation)(lstm)
    dense = Dense(6, activation='softmax')(lstm)

    model = keras.Model(inputs=x0, outputs=dense)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    metrics = ["accuracy"]
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=metrics)
    
    return model


In [ ]:
EPOCHS = 100 # number of epocs per trial
BATCH_SIZE = 80

def objective(trial):
    
    # Clear clutter from previous session graphs.
    keras.backend.clear_session()
    
    with strategy.scope():
        # Generate our trial model.
        model = create_model(trial)

        # learning rate scheduler
        scheduler = ExponentialDecay(1e-3, 400*((len(x_train)*0.8)/BATCH_SIZE), 1e-5)
        lr = LearningRateScheduler(scheduler, verbose=0)
    
        # Fit the model on the training data.
        # The TFKerasPruningCallback checks for pruning condition every epoch.
        model.fit(
            x_train,
            y_train,
            batch_size=BATCH_SIZE,
            callbacks=[TFKerasPruningCallback(trial, "val_loss")],
            epochs=EPOCHS,
            validation_data=(x_valid, y_valid),
            verbose=1,
            use_multiprocessing=True
        )

        # Evaluate the model accuracy on the validation set.
        score = model.evaluate(x_test, y_test, verbose=0)
        return score[1]

Epoch 1/2000
6/6 [==============================] - 33s 456ms/step - loss: 1.6831 - accuracy: 0.3383
Epoch 2/2000
6/6 [==============================] - 3s 455ms/step - loss: 1.4803 - accuracy: 0.4133
Epoch 3/2000
6/6 [==============================] - 3s 457ms/step - loss: 1.3348 - accuracy: 0.4783
Epoch 4/2000
6/6 [==============================] - 3s 464ms/step - loss: 1.1969 - accuracy: 0.5517
Epoch 5/2000
6/6 [==============================] - 3s 447ms/step - loss: 1.0835 - accuracy: 0.5883
Epoch 6/2000
6/6 [==============================] - 3s 448ms/step - loss: 0.9361 - accuracy: 0.6600
Epoch 7/2000
6/6 [==============================] - 3s 449ms/step - loss: 0.8190 - accuracy: 0.6950
Epoch 8/2000
6/6 [==============================] - 3s 456ms/step - loss: 0.7403 - accuracy: 0.7300
Epoch 9/2000
6/6 [==============================] - 3s 447ms/step - loss: 0.6445 - accuracy: 0.7650
Epoch 10/2000
6/6 [==============================] - 3s 448ms/step - loss: 0.5719 - accuracy: 0.781

In [ ]:
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
joblib.dump(study, "study.pkl")
study.optimize(objective, n_trials=200)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

In [1]:
# study = joblib.load("study.pkl")
# print("Best trial until now:")
# print(" Value: ", study.best_trial.value)
# print(" Params: ")
# for key, value in study.best_trial.params.items():
#     print(f"    {key}: {value}")

In [ ]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_intermediate_values(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_contour(study)